In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [2]:
df = pd.read_csv('insurance.csv')
df.sample()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low


In [3]:
df_feat = df.copy()

In [4]:
df_feat['occupation'].unique()

array(['retired', 'freelancer', 'student', 'government_job',
       'business_owner', 'unemployed', 'private_job'], dtype=object)

In [5]:
# feature 1 BMI
df_feat['bmi'] = df_feat['height']/(df_feat['weight']**2)

In [6]:
# Feature 2: Age Group
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "adult"
    elif age < 60:
        return "middle_aged"
    return "senior"

In [7]:
df_feat['age_group'] = df_feat['age'].apply(age_group)

In [8]:
# Feature 3: Lifestyle Risk
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    else:
        return "low"

In [9]:
df_feat["lifestyle_risk"] = df_feat.apply(lifestyle_risk, axis=1)

In [10]:
df_feat.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,age_group,lifestyle_risk
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,0.000109,senior,low
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,0.000179,adult,low
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,0.000508,adult,low
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,0.000130,young,medium
4,69,62.2,1.60,3.94,True,Indore,retired,High,0.000414,senior,medium


In [11]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [12]:
# Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [13]:
df_feat["city_tier"] = df_feat["city"].apply(city_tier)

In [14]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)

,income_lpa,occupation,bmi,age_group,lifestyle_risk,city_tier,insurance_premium_category
8,1.780000,retired,0.000470,senior,low,2,Medium
94,10.542289,government_job,0.000160,middle_aged,low,1,Low
74,1.880000,retired,0.000305,senior,medium,2,High
56,2.860000,student,0.000149,young,medium,1,Medium
73,2.220000,retired,0.000228,senior,medium,1,High


In [15]:
# Select features and target
X = df_feat[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]

In [16]:
X

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
0,0.000109,senior,low,2,2.92000,retired
1,0.000179,adult,low,1,34.28000,freelancer
2,0.000508,adult,low,2,36.64000,freelancer
3,0.000130,young,medium,1,3.34000,student
4,0.000414,senior,medium,2,3.94000,retired
...,...,...,...,...,...,...
95,0.000563,adult,low,2,19.64000,business_owner
96,0.000119,adult,low,1,34.01000,private_job
97,0.000487,middle_aged,low,1,44.86000,freelancer
98,0.000178,adult,low,1,28.30000,business_owner


In [17]:
y

0       High
1        Low
2        Low
3     Medium
4       High
       ...  
95       Low
96       Low
97       Low
98       Low
99       Low
Name: insurance_premium_category, Length: 100, dtype: object

In [18]:
# Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [51]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore',sparse_output=False), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

In [52]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [53]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [54]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.85

In [55]:
X_test.sample(5)

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
92,0.000471,adult,medium,2,30.000000,government_job
31,0.000701,adult,medium,2,11.770000,private_job
10,0.000342,adult,medium,1,32.780000,business_owner
69,0.000388,middle_aged,low,2,6.034487,government_job
80,0.000182,middle_aged,low,2,50.000000,unemployed


In [56]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(pipeline, f)


In [57]:
X_test.head()

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
80,0.000182,middle_aged,low,2,50.00,unemployed
84,0.000233,senior,medium,2,0.62,retired
33,0.000386,senior,low,1,1.46,retired
81,0.000236,adult,medium,2,22.19,freelancer
93,0.000293,young,low,2,1.28,student


In [33]:
pipeline.predict(pd.DataFrame([{'bmi':0.000182,'age_group':'middle_aged','lifestyle_risk':'low','city_tier':2,'income_lpa':50.00,'occupation':'unemployed'}]))[0]

'High'

In [34]:
X_train['lifestyle_risk'].unique()

array(['low', 'medium'], dtype=object)